In [1]:
import os
import csv
from collections import defaultdict
import json

# To finetune

In [60]:
DATASET_VAR = 'augm_final'
DATASET_NAME = 'dataset'
RESULTS_PATH = f'/home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/{DATASET_VAR}'
VALID_TABLE = f'{RESULTS_PATH}/valid/BirdNET_SelectionTable.txt'
TRAIN_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/train'
VALID_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/valid'
TEST_PATH = f'/home/giacomoschiavo/segments/{DATASET_NAME}/test'
TEST_TABLE = f'{RESULTS_PATH}/test/BirdNET_SelectionTable.txt'

In [61]:
print(f'python -m birdnet_analyzer.train --i {TRAIN_PATH} --o {RESULTS_PATH}/{DATASET_VAR}.tflite --batch_size 64 --threads 16 --val_split 0.01 --epochs 150 --mixup --cache_mode save --cache_file {RESULTS_PATH}/{DATASET_VAR}.npz')

python -m birdnet_analyzer.train --i /home/giacomoschiavo/segments/dataset/train --o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/augm_final/augm_final.tflite --batch_size 64 --threads 16 --val_split 0.01 --epochs 150 --mixup --cache_mode save --cache_file /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/augm_final/augm_final.npz


# To analize

In [62]:
MIN_CONF = '0.05'

In [63]:
print(f'python -m birdnet_analyzer.analyze --i {VALID_PATH} --o {RESULTS_PATH}/valid --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf {MIN_CONF} --classifier {RESULTS_PATH}/{DATASET_VAR}.tflite')

python -m birdnet_analyzer.analyze --i /home/giacomoschiavo/segments/dataset/valid --o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/augm_final/valid --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf 0.05 --classifier /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/augm_final/augm_final.tflite


In [64]:
print(f'python -m birdnet_analyzer.analyze --i {TEST_PATH} --o {RESULTS_PATH}/test --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf {MIN_CONF} --classifier {RESULTS_PATH}/{DATASET_VAR}.tflite')

python -m birdnet_analyzer.analyze --i /home/giacomoschiavo/segments/dataset/test --o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/augm_final/test --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/Labels.txt --threads 16 --combine_results --min_conf 0.05 --classifier /home/giacomoschiavo/finetuning-BirdNET/models/finetuned_2/augm_final/augm_final.tflite


# Analysis

In [65]:
with open('/home/giacomoschiavo/finetuning-BirdNET/utils/species_dict_map.json') as f:
    species_dict = json.load(f)
inv_species_dict = {value: key for key, value in species_dict.items()}

In [66]:
def get_conf_scores():
    conf_scores = {}

    #  {'Aeroplane': [(np.float32(0.0), False),
    #               (np.float32(1.3937646e-30), False),
    #               (np.float32(1.0654355e-25), False),
    #               (np.float32(0.0), False),

    with open(VALID_TABLE, 'r') as f:
        reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
        for row in reader:
            file_path = row['Begin Path']
            pred_species_name = row['Common Name']
            if row['Common Name'] in inv_species_dict:
                pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
            true_species_name = file_path.split('/')[-2]
            confidence = float(row['Confidence'])
            if pred_species_name not in conf_scores:
                conf_scores[pred_species_name] = []
            is_correct = pred_species_name == true_species_name
            conf_scores[pred_species_name].append((confidence, is_correct))
    return conf_scores
            

In [67]:
conf_scores = get_conf_scores()

In [68]:
from sklearn.metrics import f1_score
import numpy as np 

DEFAULT_THRESH = 0.15

def compute_best_thresholds(conf_scores, num_thresholds=200, min_thresh=0.001, max_thresh=0.95):
    thresholds = {}

    for species, values in conf_scores.items():
        probs, truths = zip(*values)
        probs = np.array(probs)
        truths = np.array(truths).astype(int)

        best_thresh = DEFAULT_THRESH
        best_f1 = 0.0

        for thresh in np.linspace(min_thresh, max_thresh, num_thresholds):
            preds = (probs >= thresh).astype(int)
            f1 = f1_score(truths, preds, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh

        thresholds[species] = best_thresh
        print(f"📊 {species} -> {best_thresh:.3f}, F1-score: {best_f1:.3f}")

    return thresholds



In [69]:
best_thresholds = compute_best_thresholds(conf_scores)

📊 Phylloscopus collybita_Common Chiffchaff -> 0.001, F1-score: 0.220
📊 Fringilla coelebs_Common Chaffinch -> 0.211, F1-score: 0.446
📊 Sylvia atricapilla_Eurasian Blackcap -> 0.150, F1-score: 0.000
📊 Troglodytes troglodytes_Eurasian Wren -> 0.120, F1-score: 0.436
📊 Erithacus rubecula_European Robin -> 0.068, F1-score: 0.220
📊 Turdus merula_Eurasian Blackbird -> 0.473, F1-score: 0.245
📊 Lophophanes cristatus_Crested Tit -> 0.144, F1-score: 0.182
📊 Loxia curvirostra_Common Crossbill -> 0.316, F1-score: 0.416
📊 Anthus trivialis_Tree Pipit -> 0.254, F1-score: 0.242
📊 Parus major_Great Tit -> 0.101, F1-score: 0.018
📊 Regulus ignicapilla_Common Firecrest -> 0.001, F1-score: 0.240
📊 Periparus ater_Coal Tit -> 0.592, F1-score: 0.433
📊 Certhia familiaris_Eurasian Treecreeper -> 0.797, F1-score: 0.762
📊 None -> 0.058, F1-score: 0.529
📊 Wind -> 0.087, F1-score: 0.278
📊 Coccothraustes coccothraustes_Hawfinch -> 0.077, F1-score: 0.514
📊 Regulus regulus_Goldcrest -> 0.087, F1-score: 0.945
📊 Vegetatio

In [70]:
with open(f'/home/giacomoschiavo/finetuning-BirdNET/utils/{DATASET_NAME}/true_segments_test.json') as f:
    true_segments_test = json.load(f)

In [71]:
with open(f'/home/giacomoschiavo/finetuning-BirdNET/utils/{DATASET_NAME}/true_segments_train.json') as f:
    true_segments_full = json.load(f)

In [72]:
true_segments_full.update(true_segments_test)

In [73]:
pred_segments = {}
#  {'Aeroplane': [(np.float32(0.0), False),
#               (np.float32(1.3937646e-30), False),
#               (np.float32(1.0654355e-25), False),
#               (np.float32(0.0), False),

with open(TEST_TABLE, 'r') as f:
    reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
    for row in reader:
        file_path = row['Begin Path']
        audio_name = os.path.basename(file_path)    # in test non sono presenti WABAD o augm
        only_audio_name = "_".join(audio_name.split("_")[:2]) + ".WAV"
        segm = "_".join(audio_name.split(".")[0].split("_")[-2:])
        pred_species_name = row['Common Name']
        if row['Common Name'] in inv_species_dict:
            pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
        # if len(file_path.split('/')[-2].split('_')) == 1:       # skip if None, Pecking, etc. 
        #     continue
        confidence = float(row['Confidence'])
        if only_audio_name not in pred_segments:
            pred_segments[only_audio_name] = {}
        if pred_species_name not in best_thresholds:
            best_thresholds[pred_species_name] = DEFAULT_THRESH
        if confidence >= best_thresholds[pred_species_name]:
            if segm not in pred_segments[only_audio_name]:
                pred_segments[only_audio_name][segm] = set()
            if "None" in pred_segments[only_audio_name][segm]:
                continue
            if pred_species_name == "None":
                pred_segments[only_audio_name][segm] = set(["None"])
            else:
                pred_segments[only_audio_name][segm].add(pred_species_name)
            

In [74]:
from sklearn.preprocessing import MultiLabelBinarizer

test_species_list = os.listdir(TEST_PATH)
# test_species_list = [species for species in test_species_list if len(species.split('_')) > 1]
mlb = MultiLabelBinarizer()
mlb.fit([test_species_list])

len(mlb.classes_)

20

In [75]:
true_segments = defaultdict(dict)

for species in os.listdir(TEST_PATH):
    if species not in test_species_list:
        print(species, 'ignored')
        continue
    for audio in os.listdir(os.path.join(TEST_PATH, species)):
        audio = audio.split('.')[0]
        date, time, segm1, segm2 = audio.split('_')
        audio_name = '_'.join([date, time]) + '.WAV'
        segm = '_'.join([segm1, segm2])
        if segm not in true_segments[audio_name]:
            true_segments[audio_name][segm] = []
        true_segments[audio_name][segm].extend([species])

In [76]:
for audio in true_segments.keys():
    if audio not in pred_segments:
        pred_segments[audio] = {}
    for segm in true_segments[audio].keys():
        if segm not in pred_segments[audio]:
            pred_segments[audio][segm] = []
        

In [77]:
y_pred = []
y_true = []
for audio in pred_segments:
    # sort in increasing order
    sortable_true_segments = { str(key): value for key, value in true_segments[audio].items() }
    sortable_pred_segments = { str(key): value for key, value in pred_segments[audio].items() }
    sorted_true_segments = dict(sorted(sortable_true_segments.items()))
    sorted_pred_segments = dict(sorted(sortable_pred_segments.items()))
    y_true.append(mlb.transform(sorted_true_segments.values()))     # apply transform on every label of every segment
    y_pred.append(mlb.transform(sorted_pred_segments.values()))     # apply transform on every label of every segment

y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

In [78]:
from sklearn.metrics import classification_report
import pandas as pd
report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True)

report_df = pd.DataFrame(report).T
report_df

,precision,recall,f1-score,support
Aeroplane,0.000000,0.000000,0.000000,22.0
Anthus trivialis_Tree Pipit,0.730159,0.575000,0.643357,160.0
Certhia familiaris_Eurasian Treecreeper,0.486486,0.195652,0.279070,92.0
Coccothraustes coccothraustes_Hawfinch,0.761905,0.183908,0.296296,87.0
Erithacus rubecula_European Robin,0.461453,0.742806,0.569263,556.0
Fringilla coelebs_Common Chaffinch,0.517524,0.829927,0.637511,1370.0
Lophophanes cristatus_Crested Tit,0.032258,0.130435,0.051724,23.0
Loxia curvirostra_Common Crossbill,0.458333,0.305556,0.366667,36.0
Muscicapa striata_Spotted Flycatcher,0.000000,0.000000,0.000000,173.0
None,0.819532,0.919283,0.866546,4683.0


In [79]:
with open(f'{RESULTS_PATH}/classification_report.json', 'w') as f:
    json.dump(report, f)

In [80]:
from sklearn.metrics import multilabel_confusion_matrix

mcm = multilabel_confusion_matrix(y_true, y_pred)

# Per visualizzarle in un DataFrame leggibile:
mlb_labels = mlb.classes_  # nome delle classi, già usato da te

# Costruisco un dataframe
conf_matrices = []
for idx, label in enumerate(mlb_labels):
    tn, fp, fn, tp = mcm[idx].ravel()
    conf_matrices.append({
        'Class': label,
        'TP': tp,
        'FP': fp,
        'FN': fn,
        'TN': tn
    })

conf_df = pd.DataFrame(conf_matrices)
conf_df


,Class,TP,FP,FN,TN
0,Aeroplane,0,0,22,8953
1,Anthus trivialis_Tree Pipit,92,34,68,8781
2,Certhia familiaris_Eurasian Treecreeper,18,19,74,8864
3,Coccothraustes coccothraustes_Hawfinch,16,5,71,8883
4,Erithacus rubecula_European Robin,413,482,143,7937
5,Fringilla coelebs_Common Chaffinch,1137,1060,233,6545
6,Lophophanes cristatus_Crested Tit,3,90,20,8862
7,Loxia curvirostra_Common Crossbill,11,13,25,8926
8,Muscicapa striata_Spotted Flycatcher,0,0,173,8802
9,None,4305,948,378,3344
